# DQN cartpole-experiment

## The typical imports

In [1]:
#modified from https://github.com/lbbc1117/ClassicControlDQN/blob/master/cart_pole.py

import gym
import numpy as np
from IPython.display import clear_output
import tensorflow as tf
import random
from IPython.display import Image
from IPython.core.debugger import Tracer

<img src = "dqnloop.png" width=600 > 

DQN같은경우 Replay memory를 사용한다. <br>
action선택 => 베이지안? 입실론?

## Pre-define

In [2]:
OUT_DIR = 'cartpole-experiment' # default saving directory
MAX_SCORE_QUEUE_SIZE = 100  # number of episode scores to calculate average performance
GAME = 'CartPole-v0'    # name of game

# Parameters
class options:
    def __init__(self):
        self.MAX_EPISODE = 3000 # max number of episodes iteration
        self.ACTION_DIM = 2     # number of actions one can take   : gym에서 액션 스페이스 참고
        self.OBSERVATION_DIM=4  # number of observations one can see   : 짐에서 참고 (ex, 아타리는 전체 pixel이다 따라서 cnn사용)
        self.GAMMA = 0.9        # discount factor of Q learning
        self.INIT_EPS = 1.0     # initial probability for randomly sampling action
        self.FINAL_EPS = 1e-5   # finial probability for randomly sampling action  : 로월 바운드?
        self.EPS_DECAY = 0.95   # epsilon decay rate
        self.EPS_ANNEAL_STEPS = 10 # steps interval to decay epsilon   : 언제 디케이 할거냐?
        self.LR =1e-4           # learning rate
        self.MAX_EXPERIENCE = 2000 # size of experience replay memory   : 리플레이 매모리에 스텝을 얼만큼 저장할거냐
        self.BATCH_SIZE = 256   # mini batch size  : 리플레이 메모리에서 얼만큼 패치해올거냐
        self.H1_SIZE = 128      # size of hidden layer 1
        self.H2_SIZE = 128      # size of hidden layer 2
        self.H3_SIZE = 128      # size of hidden layer 3
        
def get_options():    
    option=options()
    return option

# Agent definition

<img src = "dqn.png" width=600 >


스테이스를 내 마음대로 정의할 수도...
이미지면 cnn의 output을 여기서 input으로 사용할 수도 있따.

입력(관측) 4 노드, hidden layer 3개 각 128 노드, 출력(행동) 2 노드

딥러닝 네티워크를 효과적으로 학습하기 위해서는 Weight의 초기화를 0으로 하는 것이 아니라, 값을 주어야 하는데, 간단하게는 가우시안을 사용가능하며, 더욱 빠른 수렴 및 성능향상을 위해서 노드분포를 고려해 bias 된 값을 주는 Xavier Initialization 이용하여 초기화 하였다. 

<img src = "relu.png" width=600 >

네트워크를 학습시키는 데는 위 그림처럼 다양한 Activation Function이 존재한다. 
<br>우리는 딥러닝에서 가장 보편적으로 사용되는 ReLU를 사용하였고 이는 stochastic gradient descent의 수렴을 더욱 향상시켜준다.

In [ ]:
class QAgent:
    # A naive neural network with 3 hidden layers and relu as non-linear function.
    # 바이어스 변수를 없애면 수렴이 더 빨라지나?????????????
    def __init__(self, options): # 신경망 네트워크 정의
        self.W1 = self.weight_variable([options.OBSERVATION_DIM, options.H1_SIZE])
        self.b1 = self.bias_variable([options.H1_SIZE])
        self.W2 = self.weight_variable([options.H1_SIZE, options.H2_SIZE])
        self.b2 = self.bias_variable([options.H2_SIZE])
        self.W3 = self.weight_variable([options.H2_SIZE, options.H3_SIZE])
        self.b3 = self.bias_variable([options.H3_SIZE])
        self.W4 = self.weight_variable([options.H3_SIZE, options.ACTION_DIM])
        self.b4 = self.bias_variable([options.ACTION_DIM])
    
    # Weights initializer 
    # http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization
    def xavier_initializer(self, shape): # 노드값을 어떻게 초기화 할 것인가????  예전에는 단순히 랜덤. 이제는 점점 uniform? gaussian? 여기서는 xavier는 노드 개수를 고려해서 초기화 한다. 
        dim_sum = np.sum(shape)
        if len(shape) == 1:
            dim_sum += 1
        bound = np.sqrt(6.0 / dim_sum)
        return tf.random_uniform(shape, minval=-bound, maxval=bound)

    # Tool function to create weight variables
    def weight_variable(self, shape):
        return tf.Variable(self.xavier_initializer(shape))

    # Tool function to create bias variables
    def bias_variable(self, shape):
        return tf.Variable(self.xavier_initializer(shape))

    
    
    
    # DQN는 VALUE BASED APPROACH
    
    # Add options to graph
    def add_value_net(self, options): # Q VALUE를 어떻게 뽑아내냐
        observation = tf.placeholder(tf.float32, [None, options.OBSERVATION_DIM])
        h1 = tf.nn.relu(tf.matmul(observation, self.W1)  + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)
        h3 = tf.nn.relu(tf.matmul(h2, self.W3) + self.b3)
        Q = (tf.matmul(h3, self.W4) + self.b4) #tf.squeeze(
        return observation, Q

    # Sample action with random rate eps
    def sample_action(self, Q, feed, eps, options):
        act_values = Q.eval(feed_dict=feed)
        # 여기서는 입실론 그리디 방법을 사용, 그이외에 톰슴 샘플링, 베이지안, 등등..... 다른 것들도 할 수 있따.  EXPLORE-EXPECT 이 분야도 깊다.... 
        if random.random() <= eps: # if smaller then epsilon random select
            # action_index = env.action_space.sample()
            action_index = random.randrange(options.ACTION_DIM)
        else: # else get the maximum Q value
            action_index = np.argmax(act_values)
        action = np.zeros(options.ACTION_DIM)
        action[action_index] = 1
        return action
    
    

<img src = "greedy.jpg" width=600 >

Action의 탐색방법으로 가장 기초적인 Epsilon Greedy 방법을 사용하였다.<br>
이외 intrinsic motivation, bayesian optimization 등 탐색공간을 줄이는 방법들이 여럿 존재한다.

<img src = "dqn_loss.png" width=600 >

# DQN trainning

In [ ]:
def train(env):
    
    #1 Define placeholders to catch inputs and add options
    options = get_options() # parse the option
    agent = QAgent(options) # initiate Qagent
    
    #2 gpu usage controle (tensorflow)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True # don't hog all vRAM
    sess = tf.InteractiveSession("", config=config) #interactiveSeesion starts
    
    #3 initiate instances
    obs, Q1 = agent.add_value_net(options) # get the current observation and q value  # 현재 스테이트
    act = tf.placeholder(tf.float32, [None, options.ACTION_DIM]) # action variable  # None: 랜덤하게 다이나믹하게 왔따갔다 한다는 의미
    rwd = tf.placeholder(tf.float32, [None, ])                   # reward variable 
    next_obs, Q2 = agent.add_value_net(options) #next step observation and q value  # 다음 스테이트
    
    #4 Q value calculation and loss function calculation
    values1 = tf.reduce_sum(tf.mul(Q1, act), reduction_indices=1) # Q(s,a) prediction
    values2 = rwd + options.GAMMA * tf.reduce_max(Q2, reduction_indices=1) # r + max Q'(s',a') target
    loss = tf.reduce_mean(tf.square(values1 - values2)) # calculate loss 
    train_step = tf.train.AdamOptimizer(options.LR).minimize(loss) # can use RMSprop too   # 
    
    sess.run(tf.initialize_all_variables()) # initialize all variable
    
    
    #4 saving and loading networks   # 체크포인트를 로딩을 하는 부분... 하다가 꺼지면... 체크포인트에서 로드하면 되니까....
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("checkpoints-cartpole")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
    
    
    
    #5 Some initial local variables
    feed = {}
    eps = options.INIT_EPS
    global_step = 0
    exp_pointer = 0
    learning_finished = False
    
    #6 The replay memory
    obs_queue = np.empty([options.MAX_EXPERIENCE, options.OBSERVATION_DIM])
    act_queue = np.empty([options.MAX_EXPERIENCE, options.ACTION_DIM])
    rwd_queue = np.empty([options.MAX_EXPERIENCE])
    next_obs_queue = np.empty([options.MAX_EXPERIENCE, options.OBSERVATION_DIM])
    
    score_queue = []

    #7 The episode loop
    for i_episode in xrange(options.MAX_EPISODE):        
        observation = env.reset() #set the envirionment
        done = False
        score = 0
        sum_loss_value = 0
        
        #8 The step loop
        while not done:
            ### 잆ㄹ론 그리ㅣㄷ를 해주는 부분
            global_step += 1 # count the global_step for replay memory
            if global_step % options.EPS_ANNEAL_STEPS == 0 and eps > options.FINAL_EPS:
                eps = eps * options.EPS_DECAY # if condition meets, epsilon decays
            
            ### 리플레이 메모리에 저장해주는 부분
            obs_queue[exp_pointer] = observation # save observation in replay
            action = agent.sample_action(Q1, {obs : np.reshape(observation, (1, -1))}, eps, options) # get the action 
            act_queue[exp_pointer] = action # save action in replay 
            observation, reward, done, _ = env.step(np.argmax(action)) # next step
            
            
            score += reward #save reward every step
            reward = score  # Reward will be the accumulative score
            
            
            ### 트릭
            if done and score<200 : # druing episode, if it fail to reach 200, get hard punishment (but does not matter)
                reward = -500   # If it fails, punish hard   ### 트릭...  biased된 knolwedge 넣어주면.... 좀 더 빨리 수렴하기 위해서.. 하지만 문제마다 다르다.... 오히려 로컬 옵티마에 빠질 수 있따.
                observation = np.zeros_like(observation) # next observation is 0
            
            rwd_queue[exp_pointer] = reward # save reward in replay
            next_obs_queue[exp_pointer] = observation # next observation save in replay
    
            exp_pointer += 1 # move to next replay pointer
            if exp_pointer == options.MAX_EXPERIENCE: ### 메모리에 가득차면 2000이면.. 다시 0으로 돌아가다.. 이것도 트릭... 수렴 빨리하는 방법... 여기서는 단순히 FIFO 방식으로...  (스케쥴링과 비슷)
                exp_pointer = 0 # Refill the replay memory if it is full
    
            if global_step >= options.MAX_EXPERIENCE: # after the replay memory is full time to learn!
                rand_indexs = np.random.choice(options.MAX_EXPERIENCE, options.BATCH_SIZE) # get random index from replay  배치 256개를 램덤 선택
                feed.update({obs : obs_queue[rand_indexs]})  ## 위에 None으로 설정해줘서 가능하다....
                feed.update({act : act_queue[rand_indexs]})
                feed.update({rwd : rwd_queue[rand_indexs]})
                feed.update({next_obs : next_obs_queue[rand_indexs]})
                if not learning_finished:   # If not solved, we train and get the step loss
                    step_loss_value, _ = sess.run([loss, train_step], feed_dict = feed) #learning by loss function
                else:   # If solved, we just get the step loss
                    step_loss_value = sess.run(loss, feed_dict = feed)
                # Use sum to calculate average loss of this episode
                sum_loss_value += step_loss_value
    
        print "====== Episode {} ended with score = {}, avg_loss = {} ======".format(i_episode+1, score, sum_loss_value / score)
        score_queue.append(score)
        
        if len(score_queue) > MAX_SCORE_QUEUE_SIZE:
            score_queue.pop(0)
            if np.mean(score_queue) > 195: # The threshold of being solved
                learning_finished = True
            else:
                learning_finished = False
        if learning_finished:
            print "Testing !!!"
            
            
            
        # save progress every 100 episodes    항상 세이브 하면 좋다.
        if learning_finished and i_episode % 100 == 0:
            saver.save(sess, 'checkpoints-cartpole/' + GAME + '-dqn', global_step = global_step)
            


if __name__ == "__main__":
    env = gym.make(GAME)
    #env = gym.wrappers.Monitor(env, OUT_DIR,force=True)
    train(env)
    env.close()


[2017-05-15 08:23:30,754] Making new env: CartPole-v0


Could not find old network weights
====== Episode 1 ended with score = 14.0, avg_loss = 0.0 ======
====== Episode 2 ended with score = 18.0, avg_loss = 0.0 ======
====== Episode 3 ended with score = 30.0, avg_loss = 0.0 ======
====== Episode 4 ended with score = 18.0, avg_loss = 0.0 ======
====== Episode 5 ended with score = 20.0, avg_loss = 0.0 ======
====== Episode 6 ended with score = 12.0, avg_loss = 0.0 ======
====== Episode 7 ended with score = 18.0, avg_loss = 0.0 ======
====== Episode 8 ended with score = 17.0, avg_loss = 0.0 ======
====== Episode 9 ended with score = 11.0, avg_loss = 0.0 ======
====== Episode 10 ended with score = 9.0, avg_loss = 0.0 ======
====== Episode 11 ended with score = 9.0, avg_loss = 0.0 ======
====== Episode 12 ended with score = 12.0, avg_loss = 0.0 ======
====== Episode 13 ended with score = 10.0, avg_loss = 0.0 ======
====== Episode 14 ended with score = 11.0, avg_loss = 0.0 ======
====== Episode 15 ended with score = 9.0, avg_loss = 0.0 ======
==